In [1]:
import numpy as np
import torch 

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
# Stoke vector for the input polarized light

'''
     z    |
          |
          |________ x
      y  /
'''


s_in = np.array([[1,0,0,0]],dtype="float")



s_in_d =  torch.tensor(s_in,device=device).unsqueeze(0).unsqueeze(1)


# Flatten the lattice and create the mesh coordinate values

# Polarizer 
P_in = 0.5*np.array([[1,0,0,1],
                     [0,0,0,0],
                     [0,0,0,0],
                     [1,0,0,1]])
P_in_d =  torch.tensor(P_in,device=device)   

#theoretical pure twist result 
def director(x,z,lam):
     alpha  = (np.pi*x/lam) # Bottom Layer boundary 
     beta = np.pi/2 # Top layer boundary
     P = 2      #   No of turns or pitch
     q0 = 2*np.pi/P
     return  alpha+ np.abs(beta-alpha)*z


#Lattice configuration
X_lim = lam = 60e-6
z_lim = 14e-6
# Calculate Mueller matrix for the jth particle
N = 20
wavelength = 0.545e-6
n0 = 1
nE = 1.5
M = np.identity(4)
for j in range(N):
     print(M)
     z = j/N
     theta = np.pi/2 # change wrt to the problem
     n_ej =  np.sqrt(n0**2 + (nE**2-n0**2)*np.cos(theta)**2)
     delta = 2*np.pi*2*z_lim /N*(1/wavelength)*n0*((nE/n_ej)-1)
     phi  = np.pi/2 - director(0,z,X_lim)
     a = np.sin(2*phi)**2+(np.cos(2*phi)**2)*np.cos(delta)
     b = -np.cos(2*phi)*np.sin(delta)
     c = np.sin(2*phi)*np.cos(2*phi)*(1-np.cos(delta))
     d = np.cos(delta)
     e = -np.sin(2*phi)*np.sin(delta)
     f = np.cos(2*phi)**2+(np.sin(2*phi)**2)*np.cos(delta)

     print(a,b,c,d,e,f)
     arr = np.array([[1.0,0.0,0.0,0.0],
                   [0.0,a,b,c],
                   [0.0,-b,d,e],
                   [0.0,c,-e,f]])
     M = M @ arr

M = torch.tensor(M,device=device)
# Analyzer 
P_out = 0.5*np.array([[1,0,0,-1],
                      [0,0,0,0],
                      [0,0,0,0],
                      [-1,0,0,1]])
P_out_d =  torch.tensor(P_out,device=device)    


s_out_d = P_out_d @ M @ P_in_d @ s_in_d.T

print(s_out_d)

cuda:0
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
-0.21448510145150654 0.976727260423982 -1.4873152921047362e-16 -0.21448510145150654 -1.1961459131181928e-16 1.0
[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -2.14485101e-01  9.76727260e-01 -1.48731529e-16]
 [ 0.00000000e+00 -9.76727260e-01 -2.14485101e-01 -1.19614591e-16]
 [ 0.00000000e+00 -1.48731529e-16  1.19614591e-16  1.00000000e+00]]
-0.1847645355651712 0.9647021270621976 -0.18764826788184893 -0.21448510145150654 -0.15279380647463597 0.9702794341136646
[[ 1.          0.          0.          0.        ]
 [ 0.         -0.90262163 -0.41640768 -0.10899012]
 [ 0.          0.38737879 -0.89624701  0.21605317]
 [ 0.         -0.18764827  0.15279381  0.97027943]]
-0.09851209397047865 0.9289228256693415 -0.3569282158810619 -0.21448510145150654 -0.3018253223402955 0.8840269925189721
[[ 1.          0.          0.          0.        ]
 [ 0.          0.51463139 -0.78204857  0.3515033 ]
 [ 